## 용어
- 변수 간 상관(correlated variables): 변수들이 같은 방향으로 움직이려는 경향을 가짐. 예측 변수들끼리 서로 높은 상관성을 가질 때는 개별 계수를 해석하는 것이 어렵다.
- 다중공선성(multicollinearity): 예측변수들이 완벽하거나 거의 완벽에 가까운 상관성을 갖는다고 할 때, 회귀는 불안정하며 계산이 불가능하다.
- 교란변수(confounding variable): 중요한 예측변수이지만 회귀방정식에 누락되어 결과를 잘못되게 이끄는 변수
- 주효과(main effect): 다른 변수들과 독립된, 하나의 예측변수와 응답변수 사이의 상호 의존적인 관계

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from dmba import stepwise_selection, AIC_score

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
house = pd.read_csv('../../data/house_sales.csv', sep='\t')

## 예측변수 간 상관

In [3]:
features = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms',
              'BldgGrade', 'PropertyType', 'NbrLivingUnits',
              'SqFtFinBasement', 'YrBuilt', 'YrRenovated', 
              'NewConstruction']
label = 'AdjSalePrice'

X = pd.get_dummies(house[features], # 숫자형이 아닌 모든 컬럼들에 대해서 원핫인코딩 
                   drop_first=True) # 첫번째 카테고리는 사용하지 않음(나머지 값이 전부 0이면 첫번째 카테고리 값을 알수있다.)
X['NewConstruction'] = X['NewConstruction'].replace({False:0, True:1})
y = house[label]

# 주어진 변수 집합에 대해 적합 모델을 반환하는 함수를 정의
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(X[variables], y)
    return model

# 주어진 모델과 변수 세트에 대한 점수를 반환하는 함수를 정의
def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(y, [y.mean()]*len(y), model, df=1)
    return AIC_score(y, model.predict(X[variables]), model)

best_model, best_variables = stepwise_selection(X.columns, train_model, score_model, verbose=True)

print()
print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    print(f" {name}: {coef}")
    


#  * Bedrooms의 계수가 음수인 것을 볼 수 있다.
# => 침실 개수를 늘릴수록 그 가치가 감소한다.
# => 집이 클수록 침실이 더 많은 경향이 있으며, 침실 수보다는 주택의 크기가 주택 가격에 더 큰 영향을 준다.
#    똑같은 크기의 두 집이 있다고 하면, 작은 크기의 침실이 여러 개 있는 것을 선호하지 않는 것이 합리적이다.

Variables: SqFtTotLiving, SqFtLot, Bathrooms, Bedrooms, BldgGrade, NbrLivingUnits, SqFtFinBasement, YrBuilt, YrRenovated, NewConstruction, PropertyType_Single Family, PropertyType_Townhouse
Start: score=647988.32, constant
Step: score=633013.35, add SqFtTotLiving
Step: score=630793.74, add BldgGrade
Step: score=628230.29, add YrBuilt
Step: score=627784.16, add Bedrooms
Step: score=627602.21, add Bathrooms
Step: score=627525.65, add PropertyType_Townhouse
Step: score=627525.08, add SqFtFinBasement
Step: score=627524.98, add PropertyType_Single Family
Step: score=627524.98, unchanged None

Intercept: 6178645.017
Coefficients:
 SqFtTotLiving: 199.2775530420158
 BldgGrade: 137159.5602261976
 YrBuilt: -3565.4249392494557
 Bedrooms: -51947.383673614146
 Bathrooms: 42396.16452772052
 PropertyType_Townhouse: 84479.1620329995
 SqFtFinBasement: 7.046974967583083
 PropertyType_Single Family: 22912.05518701769


In [4]:
# Bedrooms과 상관관계가 있는 SqFtTotLiving, SqFtFinBasement, Bathrooms 제거후 계수

features = ['Bedrooms', 'BldgGrade', 'PropertyType', 'YrBuilt']
label = 'AdjSalePrice'

X = pd.get_dummies(house[features], drop_first=True)
y = house[label]

reduced_lm = LinearRegression()
reduced_lm.fit(X, y)

print(f'Intercept: {reduced_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(X.columns, reduced_lm.coef_):
    print(f' {name}: {coef}')
    
# Bedrooms에 대한 계수가 양수인 것을 확인할 수 있음.

Intercept: 4913973.344
Coefficients:
 Bedrooms: 27150.537230219703
 BldgGrade: 248997.79366212635
 YrBuilt: -3211.7448621551157
 PropertyType_Single Family: -19898.495340501973
 PropertyType_Townhouse: -47355.436873344275


## 다중공선성(Multicollinearity)
- 회귀분석에서는 다중공선성 문제를 반드시 해결해야 한다. 다중공선성이 사라질 때까지 변수를 제거해야 한다. 다중공선성이 사라질 때까지 변수를 제거해야 한다. 완전 다중공선성이 존재하는 상황에서는 회귀를 통해 제대로 된 답을 얻을수 없다.

- 완전 다중공선성: 한 예측변수가 다른 변수들의 선형 결합으로 표현 되는 것

In [5]:
# VIF 확인
def show_vif(df):
    vif = []
    for idx in range(len(df.columns)):
        vif.append(variance_inflation_factor(df.values, idx))

    vif_dataframe = pd.DataFrame()
    vif_dataframe['columns'] = df.columns
    vif_dataframe['VIF'] = vif
    return vif_dataframe

# 다중공선성 제거(vif > 10 이상인거 제거)
def remove_multicollinearity(df):
    while True:
        vif_dataframe = show_vif(df)
        
        print(f"VIF가 10 이상인 컬럼 개수: {len(vif_dataframe[vif_dataframe['VIF'] >= 10])}")
        if len(vif_dataframe[vif_dataframe['VIF'] >= 10]) == 0:
            break
        
        remove_column = vif_dataframe[vif_dataframe['VIF'] >= 10].sort_values(by='VIF', ascending=False)['columns'].reset_index(drop=True)[0]
        print(f"remove_column: {remove_column}")
        df = df.drop(remove_column, axis=1)
    return df

In [6]:
features = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms',
              'BldgGrade', 'PropertyType', 'NbrLivingUnits',
              'SqFtFinBasement', 'YrBuilt', 'YrRenovated', 
              'NewConstruction']
label = 'AdjSalePrice'

X = pd.get_dummies(house[features], # 숫자형이 아닌 모든 컬럼들에 대해서 원핫인코딩 
                   drop_first=True) # 첫번째 카테고리는 사용하지 않음(나머지 값이 전부 0이면 첫번째 카테고리 값을 알수있다.)
X['NewConstruction'] = X['NewConstruction'].replace({False:0, True:1})
y = house[label]

In [7]:
# 예측변수들끼리의 VIF 확인
show_vif(X)

,columns,VIF
0,SqFtTotLiving,34.529772
1,SqFtLot,1.235760
2,Bathrooms,26.194693
3,Bedrooms,26.298000
4,BldgGrade,135.516039
5,NbrLivingUnits,110.018883
6,SqFtFinBasement,1.986791
7,YrBuilt,714.409384
8,YrRenovated,1.085039
9,NewConstruction,1.338990


In [8]:
# VIF계산하고 최대값을 가지는 변수를 제거(VIF가 10이상일때), 이 과정을 VIF가 10이상이 없을때 까지 시행
X_new = remove_multicollinearity(X)

VIF가 10 이상인 컬럼 개수: 8
remove_column: YrBuilt
VIF가 10 이상인 컬럼 개수: 6
remove_column: BldgGrade
VIF가 10 이상인 컬럼 개수: 5
remove_column: Bathrooms
VIF가 10 이상인 컬럼 개수: 4
remove_column: Bedrooms
VIF가 10 이상인 컬럼 개수: 2
remove_column: NbrLivingUnits
VIF가 10 이상인 컬럼 개수: 0


In [9]:
# 남은 컬럼
X_new.columns

Index(['SqFtTotLiving', 'SqFtLot', 'SqFtFinBasement', 'YrRenovated',
       'NewConstruction', 'PropertyType_Single Family',
       'PropertyType_Townhouse'],
      dtype='object')

In [10]:
# 다중공선이 제거된 컬럼의 VIF 확인(해당 변수가 다른 변수와 전혀 상관 관계가 없다면 VIF=1)
show_vif(X_new)

,columns,VIF
0,SqFtTotLiving,7.972903
1,SqFtLot,1.227522
2,SqFtFinBasement,1.809953
3,YrRenovated,1.073024
4,NewConstruction,1.317730
5,PropertyType_Single Family,5.939429
6,PropertyType_Townhouse,1.293813


In [11]:
# 다중 공선성이 제거된 변수들의 회귀계수
removed_lm = LinearRegression()
removed_lm.fit(X_new, y)

print(f'Intercept: {removed_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(X_new.columns, removed_lm.coef_):
    print(f' {name}: {coef}')

Intercept: -117796.079
Coefficients:
 SqFtTotLiving: 299.1351307160863
 SqFtLot: 0.0030082082000720822
 SqFtFinBasement: 3.3747022468775723
 YrRenovated: 51.80738134944828
 NewConstruction: -69099.4706763774
 PropertyType_Single Family: 54909.69043386893
 PropertyType_Townhouse: 153803.75587451455


## 교란변수(Confounding variable)
- 회귀방정식에 중요한 변수가 포함되지 못해서 생기는 누락의 문제

In [12]:
# 위치정보를 고려하기 위해, 우편번호를 가장 싼 지역(0)에서 가장 비싼 지역(4)까지 5개의 그룹으로 분류

zip_groups = pd.DataFrame([
    *house[['ZipCode', 'AdjSalePrice']].groupby('ZipCode').apply(lambda x:{
    'ZipCode':x.iloc[0, 0],
    'count':len(x),
    'mean_price':x['AdjSalePrice'].mean()
})]).sort_values('mean_price')

zip_groups['cum_count'] = np.cumsum(zip_groups['count'])
zip_groups['ZipGroup'] = pd.qcut(zip_groups['cum_count'], 5, labels=False, retbins=False)

to_join = zip_groups[['ZipCode', 'ZipGroup']].set_index('ZipCode')
house = house.join(to_join, on='ZipCode')

In [13]:
# 그룹된화된 변수를 추가하여 선형회귀 실행

features = ['SqFtTotLiving', 'SqFtLot', 'SqFtFinBasement', 'YrRenovated', 'NewConstruction', 'PropertyType', 'ZipGroup']
label = 'AdjSalePrice'

house['ZipGroup'] = house['ZipGroup'].astype('category')

X = pd.get_dummies(house[features], drop_first=True)
y = house[label]

confounding_lm = LinearRegression()
confounding_lm.fit(X, y)

print(f'Intercept: {confounding_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(X.columns, confounding_lm.coef_):
    print(f' {name}: {coef}')
    
# ZipGroup이 중요한 변수라는 것을 확인할 수 있다.

Intercept: -180227.894
Coefficients:
 SqFtTotLiving: 252.77718153798082
 SqFtLot: 0.22595278481209569
 SqFtFinBasement: -1.5133718771428375
 YrRenovated: 39.47033339111238
 NewConstruction: -32298.629992164235
 PropertyType_Single Family: 64827.75296063482
 PropertyType_Townhouse: 98772.53857491646
 ZipGroup_1: 45189.32458097105
 ZipGroup_2: 131343.12667903677
 ZipGroup_3: 187110.7762733283
 ZipGroup_4: 358037.4345117908
